# 【課題】高エネルギー実験で生成された荷電粒子の飛跡を見つける

必要なライブラリを最初にインポートします。

### 最初に注意
**すぐ下のセルを実行すると、「セッションを再起動する」というポップアップウインドウが出てくる場合があります。これは一部のパッケージが古くて更新されていないからですが、そのまま「セッションを再起動する」をクリックして、再度同じセルを実行してください。2度目は大丈夫のはずです。それ以降のセルは普通に実行してください。**

In [ ]:
import os
import sys
import logging
import shutil
import tarfile
from google.colab import drive

drive.mount('/content/gdrive')
shutil.copy('/content/gdrive/MyDrive/qcintro.tar.gz', '.')
with tarfile.open('qcintro.tar.gz', 'r:gz') as tar:
    tar.extractall(path='/root/.local')

sys.path.append('/root/.local/lib/python3.10/site-packages')

!git clone -b branch-2024 https://github.com/kterashi/qc-workbook-lecturenotes
!cp -r qc-workbook-lecturenotes/qc_workbook /root/.local/lib/python3.10/site-packages/
!cp -r qc-workbook-lecturenotes/hepqpr /root/.local/lib/python3.10/site-packages/

%pip install qiskit-optimization
%pip install 'git+https://github.com/LAL/trackml-library.git'
%pip install dwave-qbsolv

In [ ]:
# Tested with python 3.10.11, qiskit 0.42.1, numpy 1.23.5, scipy 1.9.3
import pprint
import numpy as np
import h5py
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit
from qiskit.circuit.library import TwoLocal
from qiskit.primitives import BackendEstimator
from qiskit_algorithms.minimum_eigensolvers import VQE, NumPyMinimumEigensolver
from qiskit_algorithms.optimizers import SPSA, COBYLA
from qiskit_algorithms.gradients import ParamShiftEstimatorGradient
from qiskit.quantum_info import SparsePauliOp, Statevector
from qiskit_optimization.applications import OptimizationApplication
from qiskit_aer import AerSimulator

## データセットの生成

最初に、この課題で使う検出器データを作ります。このデータはシミュレーションで作成したものですが、実際の検出器で測定した加速器衝突実験データを模したものです。

乱数のシードを固定して、同じデータを作るようにします。この課題では`random_seed=10111`で固定してください。`density`は再構成に使う1事象あたりの飛跡の密度を表しますが、ここも0.15%で固定しておきます。

In [ ]:
from hepqpr.qallse.dsmaker import create_dataset

density = 0.0015

output_path = os.getcwd()+'/ds'
prefix = 'ds'+str(density)

metadata, path = create_dataset(
    density=density,
    output_path=output_path,
    prefix=prefix,
    random_seed=10111,
      #10111 - 17 triplets (1 track)
      # 1005 - 29 triplets (2 tracks with fake)
      # 1029 - 30 triplets (3 tracks, no fake)
    gen_doublets=True
)

下のセルを実行すると、作成したデータを整理し、データに含まれるセグメント（triplet）の数などの情報をプリントアウトします。

In [ ]:
from hepqpr.qallse import *

# ==== BUILD CONFIG
loglevel = logging.INFO

input_path = os.getcwd()+'/ds/'+prefix+'/event000001000-hits.csv'
output_path = os.getcwd()+'/ds/'+prefix+'/'

model_class = QallseD0  # model class to use
extra_config = dict()  # model config

dump_config = dict(
    output_path = os.getcwd()+'/ds/'+prefix+'/',
    prefix=prefix+'_',
    xplets_kwargs=dict(format='json', indent=3), # use json (vs "pickle") and indent the output
    qubo_kwargs=dict(w_marker=None, c_marker=None) # save the real coefficients VS generic placeholders
)

# ==== configure logging
logging.basicConfig(
    stream=sys.stderr,
    format="%(asctime)s.%(msecs)03d [%(name)-15s %(levelname)-5s] %(message)s",
    datefmt='%Y-%m-%dT%H:%M:%S')

logging.getLogger('hepqpr').setLevel(loglevel)

# ==== build model
# load data
dw = DataWrapper.from_path(input_path)
doublets = pd.read_csv(input_path.replace('-hits.csv', '-doublets.csv'))

# build model
model = model_class(dw, **extra_config)
model.build_model(doublets)

# dump model to a file
dumper.dump_model(model, **dump_config)

次のセルでデータからQUBOを作成します。

In [ ]:
import pickle
from os.path import join as path_join

from hepqpr.qallse.other.stdout_redirect import capture_stdout
from hepqpr.qallse.other.dw_timing_recorder import solver_with_timing, TimingRecord
from hepqpr.qallse.plotting import *

# ==== RUN CONFIG
nreads = 10
nseed = 1000000

loglevel = logging.INFO

input_path = os.getcwd()+'/ds/'+prefix+'/event000001000-hits.csv'
qubo_path = os.getcwd()+'/ds/'+prefix+'/'

# ==== configure logging
logging.basicConfig(
    stream=sys.stdout,
    format="%(asctime)s.%(msecs)03d [%(name)-15s %(levelname)-5s] %(message)s",
    datefmt='%Y-%m-%dT%H:%M:%S')

logging.getLogger('hepqpr').setLevel(loglevel)

# ==== build model
# load data
dw = DataWrapper.from_path(input_path)
pickle_file = prefix+'_qubo.pickle'
with open(path_join(qubo_path, pickle_file), 'rb') as f:
    Q = pickle.load(f)
#print(Q)

## ハミルトニアンの構成とVQEの実行

### QUBO

ワークブックにあるセットアップで、各セグメントを粒子飛跡の一部として採用するかフェイクとして棄却するかを考えます。具体的には、$N$個のセグメントのうち$i$番目の採用・棄却を二値変数$T_i$の値1と0に対応させ、目的関数

$$
O(b, T) = \sum_{i=1}^N a_{i} T_i + \sum_{i=1}^N \sum_{j<i}^N b_{ij} T_i T_j
$$

を最小化する$\{T_i\}$を求めます。ここで$a_i$は上で決めたセグメント$i$のスコア、$b_{ij}$はセグメント$i$と$j$のペアのスコアです。$a_i$の値が小さい（検出器中心を向いている）、かつ$b_{ij}$の値が小さい（正しい飛跡と無矛盾な）ペアを組んでいるセグメントを採用し、そうでないものを棄却するほど、$O$の値は小さくなります。採用すべきセグメントが決まれば、それに基づいてすべての粒子飛跡を再構成できるので、この最小化問題を解くことがトラッキングに対応します。

それでは、まずスコア$a_{i}$と$b_{ij}$を作成したQUBOから読み出します。

In [ ]:
# スコアの読み込み
n_max = 100

nvar = 0
key_i = []
a_score = np.zeros(n_max)
for (k1, k2), v in Q.items():
    if k1 == k2:
        a_score[nvar] = v
        key_i.append(k1)
        nvar += 1
a_score = a_score[:nvar]

b_score = np.zeros((n_max,n_max))
for (k1, k2), v in Q.items():
    if k1 != k2:
        for i in range(nvar):
            for j in range(nvar):
                if k1 == key_i[i] and k2 == key_i[j]:
                    if i < j:
                        b_score[j][i] = v
                    else:
                        b_score[i][j] = v

b_score = b_score[:nvar,:nvar]

print(f'Number of segments: {a_score.shape[0]}')
# 最初の5x5をプリント
print(a_score[:5])
print(b_score[:5, :5])

### Ising形式

QUBOの目的関数はまだハミルトニアンの形になっていない（エルミート演算子でない）ので、VQEを使ってこの問題を解くにはさらに問題を変形する必要があります。ここで$T_i$が$\{0, 1\}$のバイナリー値を持つことに着目すると、

$$
T_i = \frac{1}{2} (1 - s_i)
$$

で値$\{+1, -1\}$を持つ変数$s_i$を定義できます。次に、$\{+1, -1\}$はパウリ演算子の固有値でもあるため、$s_i$を量子ビット$i$にかかるパウリ$Z$演算子で置き換えると、$N$量子ビット系の各計算基底がセグメントの採用・棄却をエンコードする固有状態となるような目的ハミルトニアン

$$
H(h, J, s) = \sum_{i=1}^N h_i Z_i + \sum_{i=1}^N \sum_{j<i}^N J_{ij} Z_i Z_j + \text{(constant)}
$$

が得られます。これは物理を始め自然科学の様々な場面で登場するIsing模型のハミルトニアンと同じ形になっています。右辺の$constant$はハミルトニアンの定数項で、変分法において意味を持たないので以降は無視します。

### 問題1

以下のセルで、上の処方に従ってIsingハミルトニアンの係数$h_i$と$J_{ij}$を計算してください。

In [ ]:
num_qubits = nvar

coeff_h = np.zeros(num_qubits)
coeff_J = np.zeros((num_qubits, num_qubits))

##################
### EDIT BELOW ###
##################

# coeff_hとcoeff_Jをb_ijから計算してください

##################
### EDIT ABOVE ###
##################

次に、この係数をもとに、VQEに渡すハミルトニアンをSparsePauliOpとして定義します。{ref}`vqe_imp`ではSparsePauliOpは単一のパウリ積$ZXY$を表現するのに使いましたが、実はパウリ積の和も同じクラスを使って表現できます。例えば

$$
H = 0.2 IIZ + 0.3 ZZI + 0.1 ZIZ
$$

は

```python
H = SparsePauliOp(['IIZ', 'ZZI', 'ZIZ'], coeffs=[0.2, 0.3, 0.1])
```

となります。このとき、通常のQiskitの約束に従って、量子ビットの順番が右から左（一番右が第0量子ビットにかかる演算子）であることに注意してください。

### 問題2

以下のセルで、 係数が0でないパウリ積をすべて拾い出し、対応する係数の配列を作成してください。

In [ ]:
##################
### EDIT BELOW ###
##################

# 係数が0でないパウリ積をすべて拾い出し、対応する係数の配列を作成してください

pauli_products = []
coeffs = []

##################
### EDIT ABOVE ###
##################

hamiltonian = SparsePauliOp(pauli_products, coeffs=coeffs)

### 厳密対角化で基底エネルギーを求める

乱数のシードを`random_seed=10111`で固定して、飛跡の密度も`density=0.0015`に固定した場合、厳密対角化の答えは`-15.063505`になるはずです。答えの確認に使ってください。

In [ ]:
# ハミルトニアン行列を対角化して、エネルギーの最小固有値と固有ベクトルを求める
ee = NumPyMinimumEigensolver()
result_diag = ee.compute_minimum_eigenvalue(hamiltonian)

# 最小エネルギーに対応する量子ビットの組み合わせを表示
print(f'Minimum eigenvalue (diagonalization): {result_diag.eigenvalue.real}')
# 解状態を計算基底で展開し、最も確率の高い計算基底を選ぶ
optimal_segments_diag = OptimizationApplication.sample_most_likely(result_diag.eigenstate)
print(f'Optimal segments (diagonalization): {optimal_segments_diag}')

### VQEで基底エネルギーを求める


In [ ]:
backend = AerSimulator()
# Estimatorインスタンスを作る
estimator = BackendEstimator(backend)

# VQE用の変分フォームを定義。ここではTwoLocalという組み込み関数を使う
ansatz = TwoLocal(num_qubits, 'ry', 'cz', 'linear', reps=1)

# オプティマイザーを選ぶ
optimizer_name = 'SPSA'

if optimizer_name == 'SPSA':
    optimizer = SPSA(maxiter=300)
    grad = ParamShiftEstimatorGradient(estimator)

elif optimizer_name == 'COBYLA':
    optimizer = COBYLA(maxiter=500)
    grad = None

# パラメータの初期値をランダムに設定
rng = np.random.default_rng()
init = rng.uniform(0., 2. * np.pi, size=len(ansatz.parameters))

# VQEオブジェクトを作り、基底状態を探索する
vqe = VQE(estimator, ansatz, optimizer, gradient=grad, initial_point=init)
result_vqe = vqe.compute_minimum_eigenvalue(hamiltonian)

# 最適解のパラメータ値をansatzに代入し、状態ベクトルを計算する
optimal_state = Statevector(ansatz.assign_parameters(result_vqe.optimal_parameters))

# 最小エネルギーに対応する量子ビットの組み合わせを表示
print(f'Minimum eigenvalue (VQE): {result_vqe.eigenvalue.real}')
optimal_segments_vqe = OptimizationApplication.sample_most_likely(optimal_state)
print(f'Optimal segments (VQE): {optimal_segments_vqe}')

### 結果を可視化する

Trackingがうまく行っても、この答えだと0と1が並んでいるだけで面白くないですよね。正しく飛跡が見つかったかどうか目で確認するため、以下のコードを走らせてみましょう。厳密対角化の結果を図示する時は`type = diag`、VQEの結果を図示する時は`type = vqe`としてください。

正しい計算ができていれば、いくつかの情報とともに"tracks found: 1"という結果が出て、その時の飛跡の図が作られます。セルを実行したあと、このウィンドウの左端にあるフォルダアイコン（鍵マークの下）をクリックし、出てくるサブウィンドウの中の`plot-ising_[type]_found_tracks.html`をダウンロードして、ブラウザで開いてください。この図はQUBOを定義する時に使った検出器のヒット位置をビーム軸に垂直な平面に投影したものです。再構成が成功していれば、ヒットが繋がって飛跡として再構成されていることが見て取れるはずです。緑の線が実際に見つかった飛跡です。

In [ ]:
from hepqpr.qallse import DataWrapper, Qallse, TrackRecreaterD
from hepqpr.qallse.plotting import iplot_results, iplot_results_tracks
from hepqpr.qallse.utils import diff_rows

# どちらの結果をプロットするか指定する
#   diag = 厳密対角化の結果
#   vqe = VQEの結果
type = "diag"
#type = "vqe"

if type == "diag":
    optimal_segments = optimal_segments_diag
elif type == "vqe":
    optimal_segments = optimal_segments_vqe

samples = dict(zip(key_i, optimal_segments))

# 結果を取得
all_doublets = Qallse.process_sample(samples)

final_tracks, final_doublets = TrackRecreaterD().process_results(all_doublets)

#dw = DataWrapper.from_path('data/event000001000-hits.csv')
input_path = os.getcwd()+'/ds/'+prefix+'/event000001000-hits.csv'
dw = DataWrapper.from_path(input_path)

p, r, ms = dw.compute_score(final_doublets)
trackml_score = dw.compute_trackml_score(final_tracks)

print(f'SCORE  -- precision (%): {p * 100}, recall (%): {r * 100}, missing: {len(ms)}')
print(f'          tracks found: {len(final_tracks)}, trackml score (%): {trackml_score * 100}')

dims = ['x', 'y']
_, missings, _ = diff_rows(final_doublets, dw.get_real_doublets())
dout = 'plot-ising_'+type+'_found_tracks.html'
iplot_results(dw, final_doublets, missings, dims=dims, filename=dout)

### おまけ

この課題では、少数の量子ビットで実行できるように飛跡は一本だけにしています。上で指定した乱数のシードを`1029`として実行すると3本の飛跡を持つデータを作成するので、興味のある人は遊んでみてください。

色々シードを変えてデータを作って遊んでもらって良いですが、セグメントの数が多すぎるとメモリ不足を起こしてセッションがクラッシュします。

**提出するもの**
- ハミルトニアンを実装する部分のコード（問題1と問題2）